In [2]:
import folium
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm

In [3]:
data = gpd.read_file('data/서울시 격자 인구 데이터/강남서초2030대인구수 - 복사본.geojson')
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [4]:
data_gid = data[['gid', 'geometry']]

In [5]:
data_gid

,gid,geometry
0,다사579454,"MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,"MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,"MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,"MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,"MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...
9128,다사559438,"MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,"MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,"MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,"MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


## 강남구서초구정류장아침승차

In [ ]:
data1 = pd.read_csv("output/강남구서초구정류장아침승차.csv")

In [11]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","6시승차총승객수","7시승차총승객수","8시승차총승객수","9시승차총승객수"]]

In [13]:
data1["아침총승객수"] = data1["6시승차총승객수"] + data1["7시승차총승객수"]+ data1["8시승차총승객수"]+ data1["9시승차총승객수"]

In [15]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","아침총승객수"]]
data1

,표준ID,정류소명,X좌표,Y좌표,아침총승객수
0,121000001,교육개발원입구,127.038342,37.478514,287.218487
1,121000002,교육개발원입구,127.038676,37.477767,768.227273
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,1943.069444
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,607.147436
4,121000005,뱅뱅사거리,127.032367,37.487592,562.768473
...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,123.500000
1162,122000286,삼성서울병원사거리,127.082862,37.488968,276.583333
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,49.500000
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1946.032258


In [16]:
data1['bus_cnt']=1
data1['X좌표'] = data1['X좌표'].astype(float)
data1['Y좌표'] = data1['Y좌표'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

<ipython-input-16-7a876ba4cc27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['bus_cnt']=1
<ipython-input-16-7a876ba4cc27>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['X좌표'] = data1['X좌표'].astype(float)
<ipython-input-16-7a876ba4cc27>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [17]:
data1

,표준ID,정류소명,X좌표,Y좌표,아침총승객수,bus_cnt,point
0,121000001,교육개발원입구,127.038342,37.478514,287.218487,1,POINT (127.03834 37.47851)
1,121000002,교육개발원입구,127.038676,37.477767,768.227273,1,POINT (127.03868 37.47777)
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,1943.069444,1,POINT (127.03585 37.48252)
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,607.147436,1,POINT (127.03683 37.48187)
4,121000005,뱅뱅사거리,127.032367,37.487592,562.768473,1,POINT (127.03237 37.48759)
...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,123.500000,1,POINT (127.09474 37.48349)
1162,122000286,삼성서울병원사거리,127.082862,37.488968,276.583333,1,POINT (127.08286 37.48897)
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,49.500000,1,POINT (127.10154 37.46460)
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1946.032258,1,POINT (127.08768 37.49066)


In [18]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████| 1166/1166 [01:46<00:00, 10.95it/s]


In [19]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

,표준ID,정류소명,X좌표,Y좌표,아침총승객수,bus_cnt,point,gid,TL_cnt
0,121000001,교육개발원입구,127.038342,37.478514,287.218487,1,POINT (127.03834 37.47851),다사591422,1
1,121000002,교육개발원입구,127.038676,37.477767,768.227273,1,POINT (127.03868 37.47777),다사592421,1
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,1943.069444,1,POINT (127.03585 37.48252),다사589426,1
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,607.147436,1,POINT (127.03683 37.48187),다사590426,1
4,121000005,뱅뱅사거리,127.032367,37.487592,562.768473,1,POINT (127.03237 37.48759),다사586432,1
...,...,...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,123.500000,1,POINT (127.09474 37.48349),다사641427,1
1162,122000286,삼성서울병원사거리,127.082862,37.488968,276.583333,1,POINT (127.08286 37.48897),다사631433,1
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,49.500000,1,POINT (127.10154 37.46460),다사647406,1
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1946.032258,1,POINT (127.08768 37.49066),다사635435,1


In [20]:
data1 = data1[["gid","아침총승객수"]]
data1

,gid,아침총승객수
0,다사591422,287.218487
1,다사592421,768.227273
2,다사589426,1943.069444
3,다사590426,607.147436
4,다사586432,562.768473
...,...,...
1161,다사641427,123.500000
1162,다사631433,276.583333
1163,다사647406,49.500000
1164,다사635435,1946.032258


In [21]:
result1 = data1[['아침총승객수', 'gid']].groupby(['gid']).sum()

In [26]:
result1.reset_index(inplace=True)

In [28]:
result1 = result1.drop(index=0, axis=0)

In [29]:
result1

,gid,아침총승객수
1,다사540452,95.915000
2,다사541419,2123.280992
3,다사541421,933.513889
4,다사541423,957.306122
5,다사542418,200.000000
...,...,...
774,다사655420,1901.504267
775,다사658421,1662.390390
776,다사660408,25.277778
777,다사662409,47.888889


In [33]:
result1.to_csv("output/그리드 전처리 완료/서초강남버스아침승차.csv")

## 강남구서초구정류장아침하차

In [37]:
data1 = pd.read_csv("output/강남구서초구정류장아침하차.csv")

In [38]:
data1

,ARS-ID,표준ID,정류소명,X좌표,Y좌표,7시하차총승객수,8시하차총승객수,9시하차총승객수,10시하차총승객수
0,22001,121000001,교육개발원입구,127.038342,37.478514,143.907563,422.840336,237.655462,90.336134
1,22002,121000002,교육개발원입구,127.038676,37.477767,183.606061,313.151515,201.439394,120.431818
2,22003,121000003,양재역.서초문화예술회관,127.035847,37.482522,166.201389,267.368056,171.500000,101.805556
3,22004,121000004,양재역.서초문화예술회관,127.036825,37.481871,264.807692,370.653846,253.243590,191.730769
4,22005,121000005,뱅뱅사거리,127.032367,37.487592,168.369458,251.847291,159.315271,117.536946
...,...,...,...,...,...,...,...,...,...
1161,23972,122900097,수서교회.수서동성당앞,127.094743,37.483491,87.333333,63.583333,55.583333,48.583333
1162,23981,122000286,삼성서울병원사거리,127.082862,37.488968,226.000000,141.916667,137.916667,112.333333
1163,23993,122900073,강남신동아파밀리에2단지,127.101538,37.464601,29.958333,70.666667,40.416667,48.916667
1164,23994,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,530.580645,402.709677,346.483871,290.806452


In [39]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","7시하차총승객수","8시하차총승객수","9시하차총승객수","10시하차총승객수"]]

In [40]:
data1["아침총승객수"] = data1["7시하차총승객수"] + data1["8시하차총승객수"]+ data1["9시하차총승객수"]+ data1["10시하차총승객수"]

In [41]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","아침총승객수"]]
data1

,표준ID,정류소명,X좌표,Y좌표,아침총승객수
0,121000001,교육개발원입구,127.038342,37.478514,894.739496
1,121000002,교육개발원입구,127.038676,37.477767,818.628788
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,706.875000
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,1080.435897
4,121000005,뱅뱅사거리,127.032367,37.487592,697.068966
...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,255.083333
1162,122000286,삼성서울병원사거리,127.082862,37.488968,618.166667
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,189.958333
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1570.580645


In [42]:
data1['bus_cnt']=1
data1['X좌표'] = data1['X좌표'].astype(float)
data1['Y좌표'] = data1['Y좌표'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

In [43]:
data1

,표준ID,정류소명,X좌표,Y좌표,아침총승객수,bus_cnt,point
0,121000001,교육개발원입구,127.038342,37.478514,894.739496,1,POINT (127.03834 37.47851)
1,121000002,교육개발원입구,127.038676,37.477767,818.628788,1,POINT (127.03868 37.47777)
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,706.875000,1,POINT (127.03585 37.48252)
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,1080.435897,1,POINT (127.03683 37.48187)
4,121000005,뱅뱅사거리,127.032367,37.487592,697.068966,1,POINT (127.03237 37.48759)
...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,255.083333,1,POINT (127.09474 37.48349)
1162,122000286,삼성서울병원사거리,127.082862,37.488968,618.166667,1,POINT (127.08286 37.48897)
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,189.958333,1,POINT (127.10154 37.46460)
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1570.580645,1,POINT (127.08768 37.49066)


In [44]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████| 1166/1166 [02:01<00:00,  9.62it/s]


In [45]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

,표준ID,정류소명,X좌표,Y좌표,아침총승객수,bus_cnt,point,gid,TL_cnt
0,121000001,교육개발원입구,127.038342,37.478514,894.739496,1,POINT (127.03834 37.47851),다사591422,1
1,121000002,교육개발원입구,127.038676,37.477767,818.628788,1,POINT (127.03868 37.47777),다사592421,1
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,706.875000,1,POINT (127.03585 37.48252),다사589426,1
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,1080.435897,1,POINT (127.03683 37.48187),다사590426,1
4,121000005,뱅뱅사거리,127.032367,37.487592,697.068966,1,POINT (127.03237 37.48759),다사586432,1
...,...,...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,255.083333,1,POINT (127.09474 37.48349),다사641427,1
1162,122000286,삼성서울병원사거리,127.082862,37.488968,618.166667,1,POINT (127.08286 37.48897),다사631433,1
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,189.958333,1,POINT (127.10154 37.46460),다사647406,1
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1570.580645,1,POINT (127.08768 37.49066),다사635435,1


In [46]:
data1 = data1[["gid","아침총승객수"]]
data1

,gid,아침총승객수
0,다사591422,894.739496
1,다사592421,818.628788
2,다사589426,706.875000
3,다사590426,1080.435897
4,다사586432,697.068966
...,...,...
1161,다사641427,255.083333
1162,다사631433,618.166667
1163,다사647406,189.958333
1164,다사635435,1570.580645


In [47]:
result1 = data1[['아침총승객수', 'gid']].groupby(['gid']).sum()

In [48]:
result1.reset_index(inplace=True)

In [51]:
result1

,gid,아침총승객수
1,다사540452,290.648333
2,다사541419,1642.181818
3,다사541421,4109.416667
4,다사541423,1936.224490
5,다사542418,2066.666667
...,...,...
774,다사655420,438.883357
775,다사658421,495.613363
776,다사660408,138.805556
777,다사662409,171.111111


In [50]:
result1 = result1.drop(index=0, axis=0)

In [52]:
result1.to_csv("output/그리드 전처리 완료/서초강남버스아침하차.csv")

## 강남구서초구정류장저녁승차

In [53]:
data1 = pd.read_csv("output/강남구서초구정류장저녁승차.csv")

In [54]:
data1

,ARS-ID,표준ID,정류소명,X좌표,Y좌표,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수
0,22001,121000001,교육개발원입구,127.038342,37.478514,157.025210,203.605042,112.638655,95.067227
1,22002,121000002,교육개발원입구,127.038676,37.477767,409.878788,659.681818,270.151515,166.909091
2,22003,121000003,양재역.서초문화예술회관,127.035847,37.482522,330.729167,466.736111,386.590278,309.013889
3,22004,121000004,양재역.서초문화예술회관,127.036825,37.481871,271.461538,333.762821,215.237179,166.044872
4,22005,121000005,뱅뱅사거리,127.032367,37.487592,163.512315,203.635468,134.817734,112.167488
...,...,...,...,...,...,...,...,...,...
1161,23972,122900097,수서교회.수서동성당앞,127.094743,37.483491,62.666667,55.416667,15.500000,17.166667
1162,23981,122000286,삼성서울병원사거리,127.082862,37.488968,307.250000,151.916667,92.500000,71.416667
1163,23993,122900073,강남신동아파밀리에2단지,127.101538,37.464601,16.833333,20.041667,20.291667,19.791667
1164,23994,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,511.419355,324.870968,190.741935,155.516129


In [55]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","17시승차총승객수","18시승차총승객수","19시승차총승객수","20시승차총승객수"]]

In [56]:
data1["저녁총승객수"] = data1["17시승차총승객수"] + data1["18시승차총승객수"]+ data1["19시승차총승객수"]+ data1["20시승차총승객수"]

In [57]:
data1

,표준ID,정류소명,X좌표,Y좌표,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수,저녁총승객수
0,121000001,교육개발원입구,127.038342,37.478514,157.025210,203.605042,112.638655,95.067227,568.336134
1,121000002,교육개발원입구,127.038676,37.477767,409.878788,659.681818,270.151515,166.909091,1506.621212
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,330.729167,466.736111,386.590278,309.013889,1493.069444
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,271.461538,333.762821,215.237179,166.044872,986.506410
4,121000005,뱅뱅사거리,127.032367,37.487592,163.512315,203.635468,134.817734,112.167488,614.133005
...,...,...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,62.666667,55.416667,15.500000,17.166667,150.750000
1162,122000286,삼성서울병원사거리,127.082862,37.488968,307.250000,151.916667,92.500000,71.416667,623.083333
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,16.833333,20.041667,20.291667,19.791667,76.958333
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,511.419355,324.870968,190.741935,155.516129,1182.548387


In [58]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","저녁총승객수"]]
data1

,표준ID,정류소명,X좌표,Y좌표,저녁총승객수
0,121000001,교육개발원입구,127.038342,37.478514,568.336134
1,121000002,교육개발원입구,127.038676,37.477767,1506.621212
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,1493.069444
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,986.506410
4,121000005,뱅뱅사거리,127.032367,37.487592,614.133005
...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,150.750000
1162,122000286,삼성서울병원사거리,127.082862,37.488968,623.083333
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,76.958333
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1182.548387


In [59]:
data1['bus_cnt']=1
data1['X좌표'] = data1['X좌표'].astype(float)
data1['Y좌표'] = data1['Y좌표'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

<ipython-input-59-7a876ba4cc27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['bus_cnt']=1
<ipython-input-59-7a876ba4cc27>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['X좌표'] = data1['X좌표'].astype(float)
<ipython-input-59-7a876ba4cc27>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [60]:
data1

,표준ID,정류소명,X좌표,Y좌표,저녁총승객수,bus_cnt,point
0,121000001,교육개발원입구,127.038342,37.478514,568.336134,1,POINT (127.03834 37.47851)
1,121000002,교육개발원입구,127.038676,37.477767,1506.621212,1,POINT (127.03868 37.47777)
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,1493.069444,1,POINT (127.03585 37.48252)
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,986.506410,1,POINT (127.03683 37.48187)
4,121000005,뱅뱅사거리,127.032367,37.487592,614.133005,1,POINT (127.03237 37.48759)
...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,150.750000,1,POINT (127.09474 37.48349)
1162,122000286,삼성서울병원사거리,127.082862,37.488968,623.083333,1,POINT (127.08286 37.48897)
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,76.958333,1,POINT (127.10154 37.46460)
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1182.548387,1,POINT (127.08768 37.49066)


In [61]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████| 1166/1166 [02:02<00:00,  9.54it/s]


In [62]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

,표준ID,정류소명,X좌표,Y좌표,저녁총승객수,bus_cnt,point,gid,TL_cnt
0,121000001,교육개발원입구,127.038342,37.478514,568.336134,1,POINT (127.03834 37.47851),다사591422,1
1,121000002,교육개발원입구,127.038676,37.477767,1506.621212,1,POINT (127.03868 37.47777),다사592421,1
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,1493.069444,1,POINT (127.03585 37.48252),다사589426,1
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,986.506410,1,POINT (127.03683 37.48187),다사590426,1
4,121000005,뱅뱅사거리,127.032367,37.487592,614.133005,1,POINT (127.03237 37.48759),다사586432,1
...,...,...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,150.750000,1,POINT (127.09474 37.48349),다사641427,1
1162,122000286,삼성서울병원사거리,127.082862,37.488968,623.083333,1,POINT (127.08286 37.48897),다사631433,1
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,76.958333,1,POINT (127.10154 37.46460),다사647406,1
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1182.548387,1,POINT (127.08768 37.49066),다사635435,1


In [63]:
data1 = data1[["gid","저녁총승객수"]]
data1

,gid,저녁총승객수
0,다사591422,568.336134
1,다사592421,1506.621212
2,다사589426,1493.069444
3,다사590426,986.506410
4,다사586432,614.133005
...,...,...
1161,다사641427,150.750000
1162,다사631433,623.083333
1163,다사647406,76.958333
1164,다사635435,1182.548387


In [64]:
result1 = data1[['저녁총승객수', 'gid']].groupby(['gid']).sum()

In [65]:
result1.reset_index(inplace=True)

In [66]:
result1

,gid,저녁총승객수
0,Na,514.958333
1,다사540452,530.146667
2,다사541419,1891.380165
3,다사541421,3695.930556
4,다사541423,1792.285714
...,...,...
774,다사655420,678.780228
775,다사658421,635.659910
776,다사660408,74.444444
777,다사662409,145.805556


In [67]:
result1 = result1.drop(index=0, axis=0)

In [69]:
result1

,gid,저녁총승객수
1,다사540452,530.146667
2,다사541419,1891.380165
3,다사541421,3695.930556
4,다사541423,1792.285714
5,다사542418,125.583333
...,...,...
774,다사655420,678.780228
775,다사658421,635.659910
776,다사660408,74.444444
777,다사662409,145.805556


In [70]:
result1.to_csv("output/그리드 전처리 완료/서초강남버스저녁승차.csv")

## 강남구서초구정류장저녁하차

In [71]:
data1 = pd.read_csv("output/강남구서초구정류장저녁하차.csv")

In [72]:
data1

,ARS-ID,표준ID,정류소명,X좌표,Y좌표,18시하차총승객수,19시하차총승객수,20시하차총승객수,21시하차총승객수
0,22001,121000001,교육개발원입구,127.038342,37.478514,155.462185,140.571429,99.974790,98.445378
1,22002,121000002,교육개발원입구,127.038676,37.477767,358.469697,173.606061,98.053030,80.636364
2,22003,121000003,양재역.서초문화예술회관,127.035847,37.482522,229.638889,190.777778,147.277778,138.680556
3,22004,121000004,양재역.서초문화예술회관,127.036825,37.481871,1097.993590,453.198718,236.583333,194.467949
4,22005,121000005,뱅뱅사거리,127.032367,37.487592,245.325123,186.187192,140.990148,126.561576
...,...,...,...,...,...,...,...,...,...
1161,23972,122900097,수서교회.수서동성당앞,127.094743,37.483491,32.166667,15.166667,13.083333,13.000000
1162,23981,122000286,삼성서울병원사거리,127.082862,37.488968,229.083333,186.916667,161.333333,165.333333
1163,23993,122900073,강남신동아파밀리에2단지,127.101538,37.464601,210.500000,221.583333,120.541667,106.708333
1164,23994,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,483.064516,390.516129,266.258065,280.838710


In [75]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","18시하차총승객수","19시하차총승객수","20시하차총승객수","21시하차총승객수"]]

In [76]:
data1["저녁총승객수"] = data1["18시하차총승객수"] + data1["19시하차총승객수"]+ data1["20시하차총승객수"]+ data1["21시하차총승객수"]

In [77]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","저녁총승객수"]]
data1

,표준ID,정류소명,X좌표,Y좌표,저녁총승객수
0,121000001,교육개발원입구,127.038342,37.478514,494.453782
1,121000002,교육개발원입구,127.038676,37.477767,710.765152
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,706.375000
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,1982.243590
4,121000005,뱅뱅사거리,127.032367,37.487592,699.064039
...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,73.416667
1162,122000286,삼성서울병원사거리,127.082862,37.488968,742.666667
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,659.333333
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1420.677419


In [78]:
data1['bus_cnt']=1
data1['X좌표'] = data1['X좌표'].astype(float)
data1['Y좌표'] = data1['Y좌표'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

In [79]:
data1

,표준ID,정류소명,X좌표,Y좌표,저녁총승객수,bus_cnt,point
0,121000001,교육개발원입구,127.038342,37.478514,494.453782,1,POINT (127.03834 37.47851)
1,121000002,교육개발원입구,127.038676,37.477767,710.765152,1,POINT (127.03868 37.47777)
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,706.375000,1,POINT (127.03585 37.48252)
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,1982.243590,1,POINT (127.03683 37.48187)
4,121000005,뱅뱅사거리,127.032367,37.487592,699.064039,1,POINT (127.03237 37.48759)
...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,73.416667,1,POINT (127.09474 37.48349)
1162,122000286,삼성서울병원사거리,127.082862,37.488968,742.666667,1,POINT (127.08286 37.48897)
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,659.333333,1,POINT (127.10154 37.46460)
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1420.677419,1,POINT (127.08768 37.49066)


In [80]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████| 1166/1166 [02:02<00:00,  9.49it/s]


In [81]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

,표준ID,정류소명,X좌표,Y좌표,저녁총승객수,bus_cnt,point,gid,TL_cnt
0,121000001,교육개발원입구,127.038342,37.478514,494.453782,1,POINT (127.03834 37.47851),다사591422,1
1,121000002,교육개발원입구,127.038676,37.477767,710.765152,1,POINT (127.03868 37.47777),다사592421,1
2,121000003,양재역.서초문화예술회관,127.035847,37.482522,706.375000,1,POINT (127.03585 37.48252),다사589426,1
3,121000004,양재역.서초문화예술회관,127.036825,37.481871,1982.243590,1,POINT (127.03683 37.48187),다사590426,1
4,121000005,뱅뱅사거리,127.032367,37.487592,699.064039,1,POINT (127.03237 37.48759),다사586432,1
...,...,...,...,...,...,...,...,...,...
1161,122900097,수서교회.수서동성당앞,127.094743,37.483491,73.416667,1,POINT (127.09474 37.48349),다사641427,1
1162,122000286,삼성서울병원사거리,127.082862,37.488968,742.666667,1,POINT (127.08286 37.48897),다사631433,1
1163,122900073,강남신동아파밀리에2단지,127.101538,37.464601,659.333333,1,POINT (127.10154 37.46460),다사647406,1
1164,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,1420.677419,1,POINT (127.08768 37.49066),다사635435,1


In [82]:
data1 = data1[["gid","저녁총승객수"]]
data1

,gid,저녁총승객수
0,다사591422,494.453782
1,다사592421,710.765152
2,다사589426,706.375000
3,다사590426,1982.243590
4,다사586432,699.064039
...,...,...
1161,다사641427,73.416667
1162,다사631433,742.666667
1163,다사647406,659.333333
1164,다사635435,1420.677419


In [83]:
result1 = data1[['저녁총승객수', 'gid']].groupby(['gid']).sum()

In [84]:
result1.reset_index(inplace=True)

In [86]:
result1 = result1.drop(index=0, axis=0)

In [87]:
result1

,gid,저녁총승객수
1,다사540452,507.370000
2,다사541419,848.768595
3,다사541421,1874.736111
4,다사541423,1591.408163
5,다사542418,2929.583333
...,...,...
774,다사655420,1905.945235
775,다사658421,1263.244745
776,다사660408,58.083333
777,다사662409,63.138889


In [88]:
result1.to_csv("output/그리드 전처리 완료/서초강남버스저녁하차.csv")

## 강남구서초구 지하철 승하차

In [89]:
data1 = pd.read_csv("output/서초강남지하철승하차.csv")

In [90]:
data1

,Unnamed: 0,Unnamed: 0.1,역이름,위도,경도,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,5,5,강남구청역,37.517201,127.041287,8,0,2319,2329,5879,...,5963,4919,31,720,0,0,0,0,0,0
1,6,6,강남역,37.498085,127.027978,85,6,5464,10853,15861,...,24062,12861,7,4,0,0,0,0,0,0
2,12,12,개포동역,37.489132,127.066103,2,0,687,1011,3267,...,650,2034,5,58,0,0,0,0,0,0
3,20,20,고속터미널역,37.504540,127.005135,162,8,7178,6966,14260,...,19976,14774,18,387,0,0,0,0,0,0
4,28,28,교대역,37.492736,127.013863,43,0,3120,7490,14922,...,9727,8907,2,38,0,1,0,1,0,0
5,31,31,구룡역,37.487027,127.059475,0,0,360,829,1312,...,170,936,8,27,0,0,0,0,0,0
6,32,32,구반포역,37.501422,126.987447,0,0,516,236,1717,...,376,1491,0,45,0,0,0,0,0,0
7,37,37,국회의사당역,37.528136,126.917821,3,0,607,1731,1777,...,2165,1061,0,0,0,0,0,0,0,0
8,48,48,남부터미널역,37.485185,127.016191,10,0,2452,1284,8012,...,4321,6529,4,28,1,3,1,0,0,0
9,51,51,남태령역,37.464246,126.989081,46,0,453,126,824,...,90,384,0,31,0,0,0,0,0,0


In [91]:
data1 = data1[["역이름","경도","위도","06시-07시 승차인원","07시-08시 승차인원","08시-09시 승차인원","09시-10시 승차인원"]]

In [92]:
data1

,역이름,경도,위도,06시-07시 승차인원,07시-08시 승차인원,08시-09시 승차인원,09시-10시 승차인원
0,강남구청역,127.041287,37.517201,5879,13890,18347,15993
1,강남역,127.027978,37.498085,15861,36772,58759,46399
2,개포동역,127.066103,37.489132,3267,8362,8911,5240
3,고속터미널역,127.005135,37.504540,14260,48533,73258,71291
4,교대역,127.013863,37.492736,14922,31695,44067,35377
5,구룡역,127.059475,37.487027,1312,2648,3772,3317
6,구반포역,126.987447,37.501422,1717,4806,5560,3790
7,국회의사당역,126.917821,37.528136,1777,3757,6112,4945
8,남부터미널역,127.016191,37.485185,8012,25921,37710,29099
9,남태령역,126.989081,37.464246,824,2147,2747,2202


In [93]:
data1["아침총승객수"] = data1["06시-07시 승차인원"] + data1["07시-08시 승차인원"]+ data1["08시-09시 승차인원"]+ data1["09시-10시 승차인원"]

<ipython-input-93-b626f5a3266b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["아침총승객수"] = data1["06시-07시 승차인원"] + data1["07시-08시 승차인원"]+ data1["08시-09시 승차인원"]+ data1["09시-10시 승차인원"]


In [95]:
data1 = data1[["역이름","경도","위도","아침총승객수"]]
data1

,역이름,경도,위도,아침총승객수
0,강남구청역,127.041287,37.517201,54109
1,강남역,127.027978,37.498085,157791
2,개포동역,127.066103,37.489132,25780
3,고속터미널역,127.005135,37.504540,207342
4,교대역,127.013863,37.492736,126061
5,구룡역,127.059475,37.487027,11049
6,구반포역,126.987447,37.501422,15873
7,국회의사당역,126.917821,37.528136,16591
8,남부터미널역,127.016191,37.485185,100742
9,남태령역,126.989081,37.464246,7920


In [96]:
data1['sub_cnt']=1
data1['경도'] = data1['경도'].astype(float)
data1['위도'] = data1['위도'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

<ipython-input-96-19edd474c248>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sub_cnt']=1
<ipython-input-96-19edd474c248>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['경도'] = data1['경도'].astype(float)
<ipython-input-96-19edd474c248>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [97]:
data1

,역이름,경도,위도,아침총승객수,sub_cnt,point
0,강남구청역,127.041287,37.517201,54109,1,POINT (127.04129 37.51720)
1,강남역,127.027978,37.498085,157791,1,POINT (127.02798 37.49809)
2,개포동역,127.066103,37.489132,25780,1,POINT (127.06610 37.48913)
3,고속터미널역,127.005135,37.504540,207342,1,POINT (127.00513 37.50454)
4,교대역,127.013863,37.492736,126061,1,POINT (127.01386 37.49274)
5,구룡역,127.059475,37.487027,11049,1,POINT (127.05947 37.48703)
6,구반포역,126.987447,37.501422,15873,1,POINT (126.98745 37.50142)
7,국회의사당역,126.917821,37.528136,16591,1,POINT (126.91782 37.52814)
8,남부터미널역,127.016191,37.485185,100742,1,POINT (127.01619 37.48519)
9,남태령역,126.989081,37.464246,7920,1,POINT (126.98908 37.46425)


In [98]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████| 47/47 [00:05<00:00,  8.81it/s]


In [99]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

,역이름,경도,위도,아침총승객수,sub_cnt,point,gid,TL_cnt
0,강남구청역,127.041287,37.517201,54109,1,POINT (127.04129 37.51720),다사594465,1
1,강남역,127.027978,37.498085,157791,1,POINT (127.02798 37.49809),다사582444,1
2,개포동역,127.066103,37.489132,25780,1,POINT (127.06610 37.48913),다사616434,1
3,고속터미널역,127.005135,37.504540,207342,1,POINT (127.00513 37.50454),다사562451,1
4,교대역,127.013863,37.492736,126061,1,POINT (127.01386 37.49274),다사570438,1
5,구룡역,127.059475,37.487027,11049,1,POINT (127.05947 37.48703),다사610431,1
6,구반포역,126.987447,37.501422,15873,1,POINT (126.98745 37.50142),다사546448,1
7,국회의사당역,126.917821,37.528136,16591,1,POINT (126.91782 37.52814),Na,1
8,남부터미널역,127.016191,37.485185,100742,1,POINT (127.01619 37.48519),다사572429,1
9,남태령역,126.989081,37.464246,7920,1,POINT (126.98908 37.46425),다사548406,1


In [100]:
data1 = data1[["gid","아침총승객수"]]
data1

,gid,아침총승객수
0,다사594465,54109
1,다사582444,157791
2,다사616434,25780
3,다사562451,207342
4,다사570438,126061
5,다사610431,11049
6,다사546448,15873
7,Na,16591
8,다사572429,100742
9,다사548406,7920


In [101]:
result1 = data1[['아침총승객수', 'gid']].groupby(['gid']).sum()

In [102]:
result1

,아침총승객수
gid,
Na,72548
다사540449,33726
다사541420,492382
다사542431,255165
다사546448,15873
다사548406,7920
다사552432,68725
다사554450,18253
다사555425,87392


In [103]:
result1.reset_index(inplace=True)

In [104]:
result1 = result1.drop(index=0, axis=0)

In [105]:
result1

,gid,아침총승객수
1,다사540449,33726
2,다사541420,492382
3,다사542431,255165
4,다사546448,15873
5,다사548406,7920
6,다사552432,68725
7,다사554450,18253
8,다사555425,87392
9,다사562451,207342
10,다사564437,71313


In [106]:
result1.to_csv("output/그리드 전처리 완료/서초강남지하철아침승차.csv")

## 강남구서초구 지하철 승하차

In [107]:
data1 = pd.read_csv("output/서초강남지하철승하차.csv")

In [108]:
data1

,Unnamed: 0,Unnamed: 0.1,역이름,위도,경도,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,5,5,강남구청역,37.517201,127.041287,8,0,2319,2329,5879,...,5963,4919,31,720,0,0,0,0,0,0
1,6,6,강남역,37.498085,127.027978,85,6,5464,10853,15861,...,24062,12861,7,4,0,0,0,0,0,0
2,12,12,개포동역,37.489132,127.066103,2,0,687,1011,3267,...,650,2034,5,58,0,0,0,0,0,0
3,20,20,고속터미널역,37.504540,127.005135,162,8,7178,6966,14260,...,19976,14774,18,387,0,0,0,0,0,0
4,28,28,교대역,37.492736,127.013863,43,0,3120,7490,14922,...,9727,8907,2,38,0,1,0,1,0,0
5,31,31,구룡역,37.487027,127.059475,0,0,360,829,1312,...,170,936,8,27,0,0,0,0,0,0
6,32,32,구반포역,37.501422,126.987447,0,0,516,236,1717,...,376,1491,0,45,0,0,0,0,0,0
7,37,37,국회의사당역,37.528136,126.917821,3,0,607,1731,1777,...,2165,1061,0,0,0,0,0,0,0,0
8,48,48,남부터미널역,37.485185,127.016191,10,0,2452,1284,8012,...,4321,6529,4,28,1,3,1,0,0,0
9,51,51,남태령역,37.464246,126.989081,46,0,453,126,824,...,90,384,0,31,0,0,0,0,0,0


In [109]:
data1 = data1[["역이름","경도","위도","07시-08시 하차인원","08시-09시 하차인원","09시-10시 하차인원","10시-11시 하차인원"]]

In [110]:
data1

,역이름,경도,위도,07시-08시 하차인원,08시-09시 하차인원,09시-10시 하차인원,10시-11시 하차인원
0,강남구청역,127.041287,37.517201,48223,162627,136429,54003
1,강남역,127.027978,37.498085,151261,321542,311718,143770
2,개포동역,127.066103,37.489132,5893,9584,6559,4762
3,고속터미널역,127.005135,37.504540,97183,143408,144955,122455
4,교대역,127.013863,37.492736,77274,206253,167263,80836
5,구룡역,127.059475,37.487027,9114,5010,2185,1577
6,구반포역,126.987447,37.501422,3438,7021,5142,3524
7,국회의사당역,126.917821,37.528136,40486,144018,59971,20073
8,남부터미널역,127.016191,37.485185,57915,157828,77899,44551
9,남태령역,126.989081,37.464246,3799,4956,1685,1191


In [111]:
data1["아침총승객수"] = data1["07시-08시 하차인원"] + data1["08시-09시 하차인원"]+ data1["09시-10시 하차인원"]+ data1["10시-11시 하차인원"]

<ipython-input-111-8c2c1073cb5d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["아침총승객수"] = data1["07시-08시 하차인원"] + data1["08시-09시 하차인원"]+ data1["09시-10시 하차인원"]+ data1["10시-11시 하차인원"]


In [112]:
data1 = data1[["역이름","경도","위도","아침총승객수"]]
data1

,역이름,경도,위도,아침총승객수
0,강남구청역,127.041287,37.517201,401282
1,강남역,127.027978,37.498085,928291
2,개포동역,127.066103,37.489132,26798
3,고속터미널역,127.005135,37.504540,508001
4,교대역,127.013863,37.492736,531626
5,구룡역,127.059475,37.487027,17886
6,구반포역,126.987447,37.501422,19125
7,국회의사당역,126.917821,37.528136,264548
8,남부터미널역,127.016191,37.485185,338193
9,남태령역,126.989081,37.464246,11631


In [113]:
data1['sub_cnt']=1
data1['경도'] = data1['경도'].astype(float)
data1['위도'] = data1['위도'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

<ipython-input-113-19edd474c248>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sub_cnt']=1
<ipython-input-113-19edd474c248>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['경도'] = data1['경도'].astype(float)
<ipython-input-113-19edd474c248>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [114]:
data1

,역이름,경도,위도,아침총승객수,sub_cnt,point
0,강남구청역,127.041287,37.517201,401282,1,POINT (127.04129 37.51720)
1,강남역,127.027978,37.498085,928291,1,POINT (127.02798 37.49809)
2,개포동역,127.066103,37.489132,26798,1,POINT (127.06610 37.48913)
3,고속터미널역,127.005135,37.504540,508001,1,POINT (127.00513 37.50454)
4,교대역,127.013863,37.492736,531626,1,POINT (127.01386 37.49274)
5,구룡역,127.059475,37.487027,17886,1,POINT (127.05947 37.48703)
6,구반포역,126.987447,37.501422,19125,1,POINT (126.98745 37.50142)
7,국회의사당역,126.917821,37.528136,264548,1,POINT (126.91782 37.52814)
8,남부터미널역,127.016191,37.485185,338193,1,POINT (127.01619 37.48519)
9,남태령역,126.989081,37.464246,11631,1,POINT (126.98908 37.46425)


In [115]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████| 47/47 [00:07<00:00,  5.96it/s]


In [116]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

,역이름,경도,위도,아침총승객수,sub_cnt,point,gid,TL_cnt
0,강남구청역,127.041287,37.517201,401282,1,POINT (127.04129 37.51720),다사594465,1
1,강남역,127.027978,37.498085,928291,1,POINT (127.02798 37.49809),다사582444,1
2,개포동역,127.066103,37.489132,26798,1,POINT (127.06610 37.48913),다사616434,1
3,고속터미널역,127.005135,37.504540,508001,1,POINT (127.00513 37.50454),다사562451,1
4,교대역,127.013863,37.492736,531626,1,POINT (127.01386 37.49274),다사570438,1
5,구룡역,127.059475,37.487027,17886,1,POINT (127.05947 37.48703),다사610431,1
6,구반포역,126.987447,37.501422,19125,1,POINT (126.98745 37.50142),다사546448,1
7,국회의사당역,126.917821,37.528136,264548,1,POINT (126.91782 37.52814),Na,1
8,남부터미널역,127.016191,37.485185,338193,1,POINT (127.01619 37.48519),다사572429,1
9,남태령역,126.989081,37.464246,11631,1,POINT (126.98908 37.46425),다사548406,1


In [117]:
data1 = data1[["gid","아침총승객수"]]
data1

,gid,아침총승객수
0,다사594465,401282
1,다사582444,928291
2,다사616434,26798
3,다사562451,508001
4,다사570438,531626
5,다사610431,17886
6,다사546448,19125
7,Na,264548
8,다사572429,338193
9,다사548406,11631


In [118]:
result1 = data1[['아침총승객수', 'gid']].groupby(['gid']).sum()

In [119]:
result1

,아침총승객수
gid,
Na,377855
다사540449,27456
다사541420,289888
다사542431,215908
다사546448,19125
다사548406,11631
다사552432,164486
다사554450,25444
다사555425,172153


In [120]:
result1.reset_index(inplace=True)

In [121]:
result1 = result1.drop(index=0, axis=0)

In [122]:
result1

,gid,아침총승객수
1,다사540449,27456
2,다사541420,289888
3,다사542431,215908
4,다사546448,19125
5,다사548406,11631
6,다사552432,164486
7,다사554450,25444
8,다사555425,172153
9,다사562451,508001
10,다사564437,293178


In [123]:
result1.to_csv("output/그리드 전처리 완료/서초강남지하철아침하차.csv")

## 강남구서초구 지하철 승하차

In [124]:
data1 = pd.read_csv("output/서초강남지하철승하차.csv")

In [125]:
data1

,Unnamed: 0,Unnamed: 0.1,역이름,위도,경도,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,5,5,강남구청역,37.517201,127.041287,8,0,2319,2329,5879,...,5963,4919,31,720,0,0,0,0,0,0
1,6,6,강남역,37.498085,127.027978,85,6,5464,10853,15861,...,24062,12861,7,4,0,0,0,0,0,0
2,12,12,개포동역,37.489132,127.066103,2,0,687,1011,3267,...,650,2034,5,58,0,0,0,0,0,0
3,20,20,고속터미널역,37.504540,127.005135,162,8,7178,6966,14260,...,19976,14774,18,387,0,0,0,0,0,0
4,28,28,교대역,37.492736,127.013863,43,0,3120,7490,14922,...,9727,8907,2,38,0,1,0,1,0,0
5,31,31,구룡역,37.487027,127.059475,0,0,360,829,1312,...,170,936,8,27,0,0,0,0,0,0
6,32,32,구반포역,37.501422,126.987447,0,0,516,236,1717,...,376,1491,0,45,0,0,0,0,0,0
7,37,37,국회의사당역,37.528136,126.917821,3,0,607,1731,1777,...,2165,1061,0,0,0,0,0,0,0,0
8,48,48,남부터미널역,37.485185,127.016191,10,0,2452,1284,8012,...,4321,6529,4,28,1,3,1,0,0,0
9,51,51,남태령역,37.464246,126.989081,46,0,453,126,824,...,90,384,0,31,0,0,0,0,0,0


In [126]:
data1 = data1[["역이름","경도","위도","17시-18시 승차인원","18시-19시 승차인원","19시-20시 승차인원","20시-21시 승차인원"]]

In [127]:
data1

,역이름,경도,위도,17시-18시 승차인원,18시-19시 승차인원,19시-20시 승차인원,20시-21시 승차인원
0,강남구청역,127.041287,37.517201,68935,135756,77117,45361
1,강남역,127.027978,37.498085,222349,335143,203119,152318
2,개포동역,127.066103,37.489132,9663,8903,5118,3648
3,고속터미널역,127.005135,37.504540,193091,200714,151307,159775
4,교대역,127.013863,37.492736,132478,211510,92518,68407
5,구룡역,127.059475,37.487027,6309,4607,1954,1086
6,구반포역,126.987447,37.501422,6318,8344,4660,3540
7,국회의사당역,126.917821,37.528136,56927,111107,34960,20476
8,남부터미널역,127.016191,37.485185,82185,135585,61578,40318
9,남태령역,126.989081,37.464246,3693,3851,1367,1230


In [128]:
data1["저녁총승객수"] = data1["17시-18시 승차인원"] + data1["18시-19시 승차인원"]+ data1["19시-20시 승차인원"]+ data1["20시-21시 승차인원"]

<ipython-input-128-de5ba79b3c75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["저녁총승객수"] = data1["17시-18시 승차인원"] + data1["18시-19시 승차인원"]+ data1["19시-20시 승차인원"]+ data1["20시-21시 승차인원"]


In [129]:
data1 = data1[["역이름","경도","위도","저녁총승객수"]]
data1

,역이름,경도,위도,저녁총승객수
0,강남구청역,127.041287,37.517201,327169
1,강남역,127.027978,37.498085,912929
2,개포동역,127.066103,37.489132,27332
3,고속터미널역,127.005135,37.504540,704887
4,교대역,127.013863,37.492736,504913
5,구룡역,127.059475,37.487027,13956
6,구반포역,126.987447,37.501422,22862
7,국회의사당역,126.917821,37.528136,223470
8,남부터미널역,127.016191,37.485185,319666
9,남태령역,126.989081,37.464246,10141


In [130]:
data1['sub_cnt']=1
data1['경도'] = data1['경도'].astype(float)
data1['위도'] = data1['위도'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

<ipython-input-130-19edd474c248>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sub_cnt']=1
<ipython-input-130-19edd474c248>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['경도'] = data1['경도'].astype(float)
<ipython-input-130-19edd474c248>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [131]:
data1

,역이름,경도,위도,저녁총승객수,sub_cnt,point
0,강남구청역,127.041287,37.517201,327169,1,POINT (127.04129 37.51720)
1,강남역,127.027978,37.498085,912929,1,POINT (127.02798 37.49809)
2,개포동역,127.066103,37.489132,27332,1,POINT (127.06610 37.48913)
3,고속터미널역,127.005135,37.504540,704887,1,POINT (127.00513 37.50454)
4,교대역,127.013863,37.492736,504913,1,POINT (127.01386 37.49274)
5,구룡역,127.059475,37.487027,13956,1,POINT (127.05947 37.48703)
6,구반포역,126.987447,37.501422,22862,1,POINT (126.98745 37.50142)
7,국회의사당역,126.917821,37.528136,223470,1,POINT (126.91782 37.52814)
8,남부터미널역,127.016191,37.485185,319666,1,POINT (127.01619 37.48519)
9,남태령역,126.989081,37.464246,10141,1,POINT (126.98908 37.46425)


In [132]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████| 47/47 [00:05<00:00,  8.26it/s]


In [133]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

,역이름,경도,위도,저녁총승객수,sub_cnt,point,gid,TL_cnt
0,강남구청역,127.041287,37.517201,327169,1,POINT (127.04129 37.51720),다사594465,1
1,강남역,127.027978,37.498085,912929,1,POINT (127.02798 37.49809),다사582444,1
2,개포동역,127.066103,37.489132,27332,1,POINT (127.06610 37.48913),다사616434,1
3,고속터미널역,127.005135,37.504540,704887,1,POINT (127.00513 37.50454),다사562451,1
4,교대역,127.013863,37.492736,504913,1,POINT (127.01386 37.49274),다사570438,1
5,구룡역,127.059475,37.487027,13956,1,POINT (127.05947 37.48703),다사610431,1
6,구반포역,126.987447,37.501422,22862,1,POINT (126.98745 37.50142),다사546448,1
7,국회의사당역,126.917821,37.528136,223470,1,POINT (126.91782 37.52814),Na,1
8,남부터미널역,127.016191,37.485185,319666,1,POINT (127.01619 37.48519),다사572429,1
9,남태령역,126.989081,37.464246,10141,1,POINT (126.98908 37.46425),다사548406,1


In [134]:
data1 = data1[["gid","저녁총승객수"]]
data1

,gid,저녁총승객수
0,다사594465,327169
1,다사582444,912929
2,다사616434,27332
3,다사562451,704887
4,다사570438,504913
5,다사610431,13956
6,다사546448,22862
7,Na,223470
8,다사572429,319666
9,다사548406,10141


In [135]:
result1 = data1[['저녁총승객수', 'gid']].groupby(['gid']).sum()

In [136]:
result1

,저녁총승객수
gid,
Na,368478
다사540449,32063
다사541420,387570
다사542431,239794
다사546448,22862
다사548406,10141
다사552432,147331
다사554450,27760
다사555425,158923


In [137]:
result1.reset_index(inplace=True)

In [138]:
result1 = result1.drop(index=0, axis=0)

In [139]:
result1

,gid,저녁총승객수
1,다사540449,32063
2,다사541420,387570
3,다사542431,239794
4,다사546448,22862
5,다사548406,10141
6,다사552432,147331
7,다사554450,27760
8,다사555425,158923
9,다사562451,704887
10,다사564437,239588


In [140]:
result1.to_csv("output/그리드 전처리 완료/서초강남지하철저녁승차.csv")

## 강남구서초구 지하철 승하차

In [141]:
data1 = pd.read_csv("output/서초강남지하철승하차.csv")

In [142]:
data1

,Unnamed: 0,Unnamed: 0.1,역이름,위도,경도,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,5,5,강남구청역,37.517201,127.041287,8,0,2319,2329,5879,...,5963,4919,31,720,0,0,0,0,0,0
1,6,6,강남역,37.498085,127.027978,85,6,5464,10853,15861,...,24062,12861,7,4,0,0,0,0,0,0
2,12,12,개포동역,37.489132,127.066103,2,0,687,1011,3267,...,650,2034,5,58,0,0,0,0,0,0
3,20,20,고속터미널역,37.504540,127.005135,162,8,7178,6966,14260,...,19976,14774,18,387,0,0,0,0,0,0
4,28,28,교대역,37.492736,127.013863,43,0,3120,7490,14922,...,9727,8907,2,38,0,1,0,1,0,0
5,31,31,구룡역,37.487027,127.059475,0,0,360,829,1312,...,170,936,8,27,0,0,0,0,0,0
6,32,32,구반포역,37.501422,126.987447,0,0,516,236,1717,...,376,1491,0,45,0,0,0,0,0,0
7,37,37,국회의사당역,37.528136,126.917821,3,0,607,1731,1777,...,2165,1061,0,0,0,0,0,0,0,0
8,48,48,남부터미널역,37.485185,127.016191,10,0,2452,1284,8012,...,4321,6529,4,28,1,3,1,0,0,0
9,51,51,남태령역,37.464246,126.989081,46,0,453,126,824,...,90,384,0,31,0,0,0,0,0,0


In [143]:
data1 = data1[["역이름","경도","위도","18시-19시 하차인원","19시-20시 하차인원","20시-21시 하차인원","21시-22시 하차인원"]]

In [144]:
data1

,역이름,경도,위도,18시-19시 하차인원,19시-20시 하차인원,20시-21시 하차인원,21시-22시 하차인원
0,강남구청역,127.041287,37.517201,40317,24792,14116,11616
1,강남역,127.027978,37.498085,173742,99700,42059,33576
2,개포동역,127.066103,37.489132,9824,8369,4658,4244
3,고속터미널역,127.005135,37.504540,166223,104490,61196,49163
4,교대역,127.013863,37.492736,82205,48258,25356,22135
5,구룡역,127.059475,37.487027,3513,2783,1794,1595
6,구반포역,126.987447,37.501422,7631,5255,2721,2528
7,국회의사당역,126.917821,37.528136,10244,6678,3303,2963
8,남부터미널역,127.016191,37.485185,55942,35998,20899,16390
9,남태령역,126.989081,37.464246,1823,1555,981,907


In [145]:
data1["저녁총승객수"] = data1["18시-19시 하차인원"] + data1["19시-20시 하차인원"]+ data1["20시-21시 하차인원"]+ data1["21시-22시 하차인원"]

<ipython-input-145-674c835c6ca8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["저녁총승객수"] = data1["18시-19시 하차인원"] + data1["19시-20시 하차인원"]+ data1["20시-21시 하차인원"]+ data1["21시-22시 하차인원"]


In [146]:
data1 = data1[["역이름","경도","위도","저녁총승객수"]]
data1

,역이름,경도,위도,저녁총승객수
0,강남구청역,127.041287,37.517201,90841
1,강남역,127.027978,37.498085,349077
2,개포동역,127.066103,37.489132,27095
3,고속터미널역,127.005135,37.504540,381072
4,교대역,127.013863,37.492736,177954
5,구룡역,127.059475,37.487027,9685
6,구반포역,126.987447,37.501422,18135
7,국회의사당역,126.917821,37.528136,23188
8,남부터미널역,127.016191,37.485185,129229
9,남태령역,126.989081,37.464246,5266


In [147]:
data1['sub_cnt']=1
data1['경도'] = data1['경도'].astype(float)
data1['위도'] = data1['위도'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

<ipython-input-147-19edd474c248>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sub_cnt']=1
<ipython-input-147-19edd474c248>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['경도'] = data1['경도'].astype(float)
<ipython-input-147-19edd474c248>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [148]:
data1

,역이름,경도,위도,저녁총승객수,sub_cnt,point
0,강남구청역,127.041287,37.517201,90841,1,POINT (127.04129 37.51720)
1,강남역,127.027978,37.498085,349077,1,POINT (127.02798 37.49809)
2,개포동역,127.066103,37.489132,27095,1,POINT (127.06610 37.48913)
3,고속터미널역,127.005135,37.504540,381072,1,POINT (127.00513 37.50454)
4,교대역,127.013863,37.492736,177954,1,POINT (127.01386 37.49274)
5,구룡역,127.059475,37.487027,9685,1,POINT (127.05947 37.48703)
6,구반포역,126.987447,37.501422,18135,1,POINT (126.98745 37.50142)
7,국회의사당역,126.917821,37.528136,23188,1,POINT (126.91782 37.52814)
8,남부터미널역,127.016191,37.485185,129229,1,POINT (127.01619 37.48519)
9,남태령역,126.989081,37.464246,5266,1,POINT (126.98908 37.46425)


In [149]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████| 47/47 [00:05<00:00,  8.10it/s]


In [150]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

,역이름,경도,위도,저녁총승객수,sub_cnt,point,gid,TL_cnt
0,강남구청역,127.041287,37.517201,90841,1,POINT (127.04129 37.51720),다사594465,1
1,강남역,127.027978,37.498085,349077,1,POINT (127.02798 37.49809),다사582444,1
2,개포동역,127.066103,37.489132,27095,1,POINT (127.06610 37.48913),다사616434,1
3,고속터미널역,127.005135,37.504540,381072,1,POINT (127.00513 37.50454),다사562451,1
4,교대역,127.013863,37.492736,177954,1,POINT (127.01386 37.49274),다사570438,1
5,구룡역,127.059475,37.487027,9685,1,POINT (127.05947 37.48703),다사610431,1
6,구반포역,126.987447,37.501422,18135,1,POINT (126.98745 37.50142),다사546448,1
7,국회의사당역,126.917821,37.528136,23188,1,POINT (126.91782 37.52814),Na,1
8,남부터미널역,127.016191,37.485185,129229,1,POINT (127.01619 37.48519),다사572429,1
9,남태령역,126.989081,37.464246,5266,1,POINT (126.98908 37.46425),다사548406,1


In [151]:
data1 = data1[["gid","저녁총승객수"]]
data1

,gid,저녁총승객수
0,다사594465,90841
1,다사582444,349077
2,다사616434,27095
3,다사562451,381072
4,다사570438,177954
5,다사610431,9685
6,다사546448,18135
7,Na,23188
8,다사572429,129229
9,다사548406,5266


In [152]:
result1 = data1[['저녁총승객수', 'gid']].groupby(['gid']).sum()

In [153]:
result1

,저녁총승객수
gid,
Na,124945
다사540449,31264
다사541420,557818
다사542431,288908
다사546448,18135
다사548406,5266
다사552432,69086
다사554450,18702
다사555425,96834


In [154]:
result1.reset_index(inplace=True)

In [155]:
result1 = result1.drop(index=0, axis=0)

In [156]:
result1

,gid,저녁총승객수
1,다사540449,31264
2,다사541420,557818
3,다사542431,288908
4,다사546448,18135
5,다사548406,5266
6,다사552432,69086
7,다사554450,18702
8,다사555425,96834
9,다사562451,381072
10,다사564437,54088


In [157]:
result1.to_csv("output/그리드 전처리 완료/서초강남지하철저녁하차.csv")